In [1]:
import pandas as pd
import logging
import string

# ------------------------------------------------------------------------------
# Set up logging
# ------------------------------------------------------------------------------
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ------------------------------------------------------------------------------
# Configuration: alternate names for product_id
# ------------------------------------------------------------------------------
ALTERNATE_PRODUCT_ID_COLUMNS = ["item_nbr", "item_no_nbr", "item_id"]


# ------------------------------------------------------------------------------
# Helper Function to Standardize "to_be_dropped"
# ------------------------------------------------------------------------------
def standardize_to_be_dropped(value):
    """
    Remove punctuation, convert to lowercase, and trim whitespace.
    """
    val_str = str(value)
    # Remove punctuation
    val_str_no_punc = val_str.translate(str.maketrans("", "", string.punctuation))
    # Lowercase and strip
    return val_str_no_punc.lower().strip()


# ------------------------------------------------------------------------------
# Main Processing Function
# ------------------------------------------------------------------------------
def process_laundry_files(file_paths):
    """
    Processes each Excel file in file_paths:
      - Reads the file.
      - Reconciles the product_id column (renames alternate columns if needed).
      - Standardizes the 'to_be_dropped' column and logs any unexpected values.
      - Filters rows where 'to_be_dropped' (standardized) equals 'no'.
      - Adds a 'data_source' column with the file name.

    Returns two DataFrames:
      1. full_df: The union of all columns from the files (with data_source).
      2. minimal_df: Only 'product_id' and 'data_source'.
    """
    full_frames = []
    minimal_frames = []

    for file_path in file_paths:
        try:
            df = pd.read_excel(file_path)
            logger.info(f"Successfully read file: {file_path}")
        except Exception as e:
            logger.error(f"Error reading file {file_path}: {e}")
            continue

        # --- Reconcile product_id ---
        if "product_id" not in df.columns:
            found_alternate = False
            for alt in ALTERNATE_PRODUCT_ID_COLUMNS:
                if alt in df.columns:
                    logger.warning(
                        f"File {file_path}: No 'product_id' column found. Renaming '{alt}' to 'product_id'."
                    )
                    df.rename(columns={alt: "product_id"}, inplace=True)
                    found_alternate = True
                    break
            if not found_alternate:
                logger.warning(f"File {file_path}: No 'product_id' column found. Skipping file.")
                continue

        # --- Standardize and check 'to_be_dropped' ---
        if "to_be_dropped" not in df.columns:
            logger.warning(f"File {file_path}: 'to_be_dropped' column not found. Skipping file.")
            continue

        df["to_be_dropped_std"] = df["to_be_dropped"].apply(standardize_to_be_dropped)
        # Log unexpected values (only 'yes' or 'no' expected)
        unexpected_vals = df.loc[
            ~df["to_be_dropped_std"].isin(["yes", "no"]), "to_be_dropped_std"
        ].unique()
        if len(unexpected_vals) > 0:
            logger.warning(
                f"File {file_path}: Unexpected values in 'to_be_dropped': {list(unexpected_vals)}"
            )

        # --- Filter rows where to_be_dropped is 'no'
        df_filtered = df[df["to_be_dropped_std"] == "no"].copy()
        if df_filtered.empty:
            logger.info(f"File {file_path}: No rows with 'to_be_dropped' == 'no' after filtering.")
            continue

        # --- Add data_source column ---
        df_filtered["data_source"] = file_path

        # --- For full union output, drop the internal standardization column ---
        if "to_be_dropped_std" in df_filtered.columns:
            df_filtered.drop(columns=["to_be_dropped_std"], inplace=True)

        full_frames.append(df_filtered)

        # --- Create minimal output (only product_id and data_source) ---
        minimal_df = df_filtered[["product_id", "data_source"]].copy()
        minimal_frames.append(minimal_df)

    # --- Concatenate all processed DataFrames ---
    if full_frames:
        full_df = pd.concat(full_frames, ignore_index=True, sort=False)
    else:
        full_df = pd.DataFrame()

    if minimal_frames:
        minimal_df = pd.concat(minimal_frames, ignore_index=True)
    else:
        minimal_df = pd.DataFrame(columns=["product_id", "data_source"])

    return full_df, minimal_df

In [2]:
# Define the three Excel file paths
file_paths = [
    "Fabric Enhancers.xlsx",
    "Laundry Bleach.xlsx",
    "Laundry.xlsx",
]

# Process the files
full_output_df, minimal_output_df = process_laundry_files(file_paths)

# Write out the minimal file (only product_id and data_source)
minimal_output_file = "combined_product_ids.csv"
try:
    minimal_output_df.to_csv(minimal_output_file, index=False)
    logger.info(f"Minimal output written to: {minimal_output_file}")
except Exception as e:
    logger.error(f"Error writing {minimal_output_file}: {e}")

# Write out the full union file (all unique columns plus data_source)
full_output_file = "combined_full.csv"
try:
    full_output_df.to_csv(full_output_file, index=False)
    logger.info(f"Full union output written to: {full_output_file}")
except Exception as e:
    logger.error(f"Error writing {full_output_file}: {e}")

# Optionally, print the shapes of the outputs for verification
print("Minimal output shape:", minimal_output_df.shape)
print("Full output shape:", full_output_df.shape)

Minimal output shape: (203, 2)
Full output shape: (203, 15)
